<a href="https://colab.research.google.com/github/vinay-jose/rag-nbs/blob/main/IR_Baseline_for_RAG_MVP%2B%2B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -Uq lancedb ir_datasets tantivy sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.4/347.4 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 36.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.5/25.5 MB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 14.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 53.4 MB/s eta 0:00:00
  Preparing meta

In [ ]:
# !pip install -Uqq

In [ ]:
import ir_datasets
dataset = ir_datasets.load('cord19/trec-covid')

In [ ]:
dataset.docs_count()

192509

In [ ]:
import pandas as pd
pd.DataFrame(dataset.docs_iter()).head()

[INFO] [starting] building docstore
[INFO] If you have a local copy of https://ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com/2020-07-16/metadata.csv, you can symlink it here to avoid downloading it again: /root/.ir_datasets/downloads/80d664e496b8b7e50a39c6f6bb92e0ef
[INFO] [starting] https://ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com/2020-07-16/metadata.csv
docs_iter:   0%|                                    | 0/192509 [00:00<?, ?doc/s]
https://ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com/2020-07-16/metadata.csv: 0.0%| 0.00/269M [00:00<?, ?B/s]
https://ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com/2020-07-16/metadata.csv: 0.0%| 57.3k/269M [00:00<09:40, 464kB/s]
https://ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com/2020-07-16/metadata.csv: 0.1%| 279k/269M [00:00<04:17, 1.05MB/s]
https://ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com/2020-07-16/metadata.csv: 0.4%| 1.20M/269M [00:00<01:30, 2.95MB/s]
https://ai2-semanticscholar-cord-

,doc_id,title,doi,date,abstract
0,ug7v899j,Clinical features of culture-proven Mycoplasma...,10.1186/1471-2334-1-6,2001-07-04,OBJECTIVE: This retrospective chart review des...
1,02tnwd4m,Nitric oxide: a pro-inflammatory mediator in l...,10.1186/rr14,2000-08-15,Inflammatory diseases of the respiratory tract...
2,ejv2xln0,Surfactant protein-D and pulmonary host defense,10.1186/rr19,2000-08-25,Surfactant protein-D (SP-D) participates in th...
3,2b73a28n,Role of endothelin-1 in lung disease,10.1186/rr44,2001-02-22,Endothelin-1 (ET-1) is a 21 amino acid peptide...
4,9785vg6d,Gene expression in epithelial cells in respons...,10.1186/rr61,2001-05-11,Respiratory syncytial virus (RSV) and pneumoni...


In [ ]:
dataset.docs_cls()._fields

('doc_id', 'title', 'doi', 'date', 'abstract')

In [ ]:
dataset.docs_cls().__annotations__

{'doc_id': str, 'title': str, 'doi': str, 'date': str, 'abstract': str}

In [ ]:
dataset.queries_count()

50

In [ ]:
pd.DataFrame(dataset.queries_iter())

[INFO] [starting] https://ir.nist.gov/covidSubmit/data/topics-rnd5.xml
[INFO] [finished] https://ir.nist.gov/covidSubmit/data/topics-rnd5.xml: [00:00] [18.7kB] [33.1MB/s]


,query_id,title,description,narrative
0,1,coronavirus origin,what is the origin of COVID-19,seeking range of information about the SARS-Co...
1,2,coronavirus response to weather changes,how does the coronavirus respond to changes in...,seeking range of information about the SARS-Co...
2,3,coronavirus immunity,will SARS-CoV2 infected people develop immunit...,seeking studies of immunity developed due to i...
3,4,how do people die from the coronavirus,what causes death from Covid-19?,Studies looking at mechanisms of death from Co...
4,5,animal models of COVID-19,what drugs have been active against SARS-CoV o...,Papers that describe the results of testing d...
5,6,coronavirus test rapid testing,what types of rapid testing for Covid-19 have ...,Looking for studies identifying ways to diagno...
6,7,serological tests for coronavirus,are there serological tests that detect antibo...,Looking for assays that measure immune respons...
7,8,coronavirus under reporting,how has lack of testing availability led to un...,Looking for studies answering questions of imp...
8,9,coronavirus in Canada,how has COVID-19 affected Canada,"seeking data related to infections (confirm, s..."
9,10,coronavirus social distancing impact,has social distancing had an impact on slowing...,seeking specific information on studies that h...


In [ ]:
pd.DataFrame(dataset.queries_iter()).description[1]

'how does the coronavirus respond to changes in the weather'

In [ ]:
pd.DataFrame(dataset.qrels_iter())

[INFO] [starting] https://ir.nist.gov/covidSubmit/data/qrels-covid_d5_j0.5-5.txt
[INFO] [finished] https://ir.nist.gov/covidSubmit/data/qrels-covid_d5_j0.5-5.txt: [00:00] [1.14MB] [8.70MB/s]


,query_id,doc_id,relevance,iteration
0,1,005b2j4b,2,4.5
1,1,00fmeepz,1,4
2,1,010vptx3,2,0.5
3,1,0194oljo,1,2.5
4,1,021q9884,1,4
...,...,...,...,...
69313,50,zvop8bxh,2,5
69314,50,zwf26o63,1,5
69315,50,zwsvlnwe,0,5
69316,50,zxr01yln,1,5


In [ ]:
dataset.qrels_defs()

{2: 'Relevant: the article is fully responsive to the information need as expressed by the topic, i.e. answers the Question in the topic. The article need not contain all information on the topic, but must, on its own, provide an answer to the question.',
 1: 'Partially Relevant: the article answers part of the question but would need to be combined with other information to get a complete answer.',
 0: 'Not Relevant: everything else.'}

In [ ]:
dataset.qrels_cls().__annotations__

{'query_id': str, 'doc_id': str, 'relevance': int, 'iteration': str}

In [ ]:
import lancedb

uri = "data/sample-lancedb"
db = lancedb.connect(uri)


In [ ]:
tbl = db.create_table("table_from_df", data=pd.DataFrame(dataset.docs_iter()))

In [ ]:
tbl.head()

pyarrow.Table
doc_id: string
title: string
doi: string
date: string
abstract: string
----
doc_id: [["ug7v899j","02tnwd4m","ejv2xln0","2b73a28n","9785vg6d"]]
title: [["Clinical features of culture-proven Mycoplasma pneumoniae infections at King Abdulaziz University Hospital, Jeddah, Saudi Arabia","Nitric oxide: a pro-inflammatory mediator in lung disease?","Surfactant protein-D and pulmonary host defense","Role of endothelin-1 in lung disease","Gene expression in epithelial cells in response to pneumovirus infection"]]
doi: [["10.1186/1471-2334-1-6","10.1186/rr14","10.1186/rr19","10.1186/rr44","10.1186/rr61"]]
date: [["2001-07-04","2000-08-15","2000-08-25","2001-02-22","2001-05-11"]]
abstract: [["OBJECTIVE: This retrospective chart review describes the epidemiology and clinical features of 40 patients with culture-proven Mycoplasma pneumoniae infections at King Abdulaziz University Hospital, Jeddah, Saudi Arabia. METHODS: Patients with positive M. pneumoniae cultures from respiratory sp

In [ ]:
tbl.create_fts_index("abstract", replace=True)

In [ ]:
q = pd.DataFrame(dataset.queries_iter()).description[1]
q, type(q)

('how does the coronavirus respond to changes in the weather', str)

In [ ]:
from lancedb.rerankers import ColbertReranker

In [ ]:
reranker = ColbertReranker(column="abstract")
results_baseline = tbl.search(q, query_type="fts").limit(10).rerank(reranker=reranker).to_list()

tokenizer_config.json:   0%|          | 0.00/405 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [ ]:
len(results_baseline), results_baseline[0]

(10,
 {'doc_id': '9svrz0vj',
  'title': 'Susceptible supply limits the role of climate in the early SARS-CoV-2 pandemic',
  'doi': '10.1126/science.abc2535',
  'date': '2020-07-10',
  'abstract': 'Preliminary evidence suggests that climate may modulate the transmission of SARS-CoV-2. Yet it remains unclear whether seasonal and geographic variations in climate can substantially alter the pandemic trajectory, given high susceptibility is a core driver. Here, we use a climate-dependent epidemic model to simulate the SARS-CoV-2 pandemic probing different scenarios based on known coronavirus biology. We find that while variations in weather may be important for endemic infections, during the pandemic stage of an emerging pathogen the climate drives only modest changes to pandemic size. A preliminary analysis of non-pharmaceutical control measures indicates that they may moderate the pandemic-climate interaction via susceptible depletion. Our findings suggest, without effective control measu

In [ ]:
from lancedb.pydantic import LanceModel, Vector
from lancedb.embeddings import get_registry

model = get_registry().get("sentence-transformers").create(name="BAAI/bge-small-en-v1.5")

class Document(LanceModel):
    abstract_embed: Vector(384) = model.VectorField()
    abstract: str = model.SourceField()
    doc_id: str
    title: str
    doi: str
    date: str


db = lancedb.connect("data/vectordb")
tbl = db.create_table("vectors", schema=Document)

In [ ]:
# db.drop_table("vectors")

In [ ]:
tbl.add(pd.DataFrame(dataset.docs_iter()[:100]))

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
results_biencoder = tbl.search(q, query_type="vector").limit(10).rerank(reranker=reranker).to_list()

In [ ]:
len(results_baseline), results_baseline[0]

(10,
 {'doc_id': '9svrz0vj',
  'title': 'Susceptible supply limits the role of climate in the early SARS-CoV-2 pandemic',
  'doi': '10.1126/science.abc2535',
  'date': '2020-07-10',
  'abstract': 'Preliminary evidence suggests that climate may modulate the transmission of SARS-CoV-2. Yet it remains unclear whether seasonal and geographic variations in climate can substantially alter the pandemic trajectory, given high susceptibility is a core driver. Here, we use a climate-dependent epidemic model to simulate the SARS-CoV-2 pandemic probing different scenarios based on known coronavirus biology. We find that while variations in weather may be important for endemic infections, during the pandemic stage of an emerging pathogen the climate drives only modest changes to pandemic size. A preliminary analysis of non-pharmaceutical control measures indicates that they may moderate the pandemic-climate interaction via susceptible depletion. Our findings suggest, without effective control measu